<a href="https://colab.research.google.com/github/hyydrra/Decision-Tree/blob/master/IS_hw2_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import collections
import random

In [ ]:

  import numpy as np
  def entt(yp,yn,x1p,x1n,x2p,x2n  )
    if y1==0 or y0==0:
      parent=0
    else:
      parent = -((y0/(y1+y0))*np.log2(y0/(y1+y0))) -((y1/(y1+y0))*np.log2(y1/(y1+y0)))

    if x11==0 or x10==0:
      child1=0
    else:
      child1 = (-((x11/(x11+x10))*np.log2(x11/(x11+x10))) -((x10/(x11+x10))*np.log2(x10/(x11+x10))))

    if x21==0 or x20==0:
      child2=0
    else:
      child2 = (-((x21/(x21+x20))*np.log2(x21/(x21+x20))) -((x20/(x21+x20))*np.log2(x20/(x21+x20))))
    ent = parent -(((x11+x10)/(y1+y0))*child1) -(((x21+x20)/(y1+y0))*child2)
    return ent 

In [ ]:
data=pd.read_csv('/content/arrhythmia.data.csv')

In [ ]:
price=data.iloc[0,:]
del_ind=[]
for x in range(data.shape[0]):
  for y in range(data.shape[1]):
    if data.iloc[x,y]=="?":
      del_ind.append(x)
      break
fdata=data.drop(data.index[del_ind])
fdata=fdata.reset_index()
fdata=fdata.drop("index", axis=1)

classes=set()
for i in range(fdata.shape[0]):
  classes.add(fdata.iloc[i,-1])
classes=list(classes)

In [ ]:
discrete=[]
continuous=[]
for i in range(fdata.shape[1]):
  disc=1
  for j in range(fdata.shape[0]):
    if fdata.iloc[j,i] !=1:
      if fdata.iloc[j,i] !=0:
        disc=0
  if disc==0:
    continuous.append(i)
  else :
    discrete.append(i)
discrete
continuous.pop(-1)
disc_data = fdata.drop(fdata.columns[continuous], axis = 1)
continuous_data = fdata.drop(fdata.columns[discrete], axis = 1)

In [ ]:
def counter(attribute):
  class1=0
  class0=0
  for x in attribute:
    if x==1:
      class1 = class1+1
    else:
      class0 = class0+1
  return class1,class0
    
def entropy_cont(attribute,ind):
  part1 = attribute[0:ind]
  part2 = attribute[ind:]
  y1,y0 = counter(attribute)
  x11,x10 = counter(part1)
  x21,x20 = counter(part2)

  if y1==0 or y0==0:
    parent=0
  else:
    parent = -((y0/(y1+y0))*np.log2(y0/(y1+y0))) -((y1/(y1+y0))*np.log2(y1/(y1+y0)))

  if x11==0 or x10==0:
    child1=0
  else:
    child1 = (-((x11/(x11+x10))*np.log2(x11/(x11+x10))) -((x10/(x11+x10))*np.log2(x10/(x11+x10))))

  if x21==0 or x20==0:
    child2=0
  else:
    child2 = (-((x21/(x21+x20))*np.log2(x21/(x21+x20))) -((x20/(x21+x20))*np.log2(x20/(x21+x20))))
  entropy = parent -(((x11+x10)/(y1+y0))*child1) -(((x21+x20)/(y1+y0))*child2)
  return entropy


In [ ]:
def maxmin(dataframe,coloumn):
  Min=min(list(dataframe.iloc[:,coloumn]))
  Max=max(list(dataframe.iloc[:,coloumn]))
  return Max,Min
 
def div_points(dataframe,coloumn,num):
  points=[]
  Max,Min = maxmin(dataframe,coloumn)
  l = (Max-Min)/num
  for i in range(num+1):
    points.append(Min+i*l)
  return points

Brakin_points for continuous data

In [ ]:
points = 2*np.array(list(range(1,33)))
break_points={}
all_classes=[]
for k in range(len(classes)):
  c=classes[k]
  ps=[]
  for a in range(continuous_data.shape[1]):
    class1 = list(continuous_data.iloc[:,-1])
    for x in range(len(class1)):
      if class1[x]!=c:
        class1[x]=0
      else:
        class1[x]=1
  

    temp=[]
    for i in range(continuous_data.shape[0]):
      temp.append([])
      temp[i].append(float(continuous_data.iloc[i,a]))
      temp[i].append(float(class1[i]))
    sorted_temp = np.array(sorted(temp,key=lambda l:l[0]))
    maxent=0
    for p in points:
      ent = entropy_cont(sorted_temp[:,1],p)
      if ent> maxent:
        maxent=ent
        maxp=p

    val = (sorted_temp[maxp-1,0] + sorted_temp[maxp+1,0])/2
    ps.append(val)
  break_points[k]=ps
  all_classes.append(class1)

In [ ]:
classes

[1, 2, 3, 4, 5, 6, 8, 9, 10, 14, 16]

Training the tree

In [ ]:
def counter1(indexes,num_class):
  class1=0
  class2=0
  for x in indexes:
    if all_classes[num_class][x]==1:
      class1=class1+1
    else:
      class2=class2+1
  return class1,class2

def Entropy(nodedata,attri,isdisc,br_point,num_class):
  part1=[]
  part2=[]
  if isdisc:
    for i in nodedata:
      if float(disc_data.iloc[i,attri])<br_point:
        part1.append(i)
      else:
        part2.append(i)
  else:
    for i in nodedata:
      if float(continuous_data.iloc[i,attri])<br_point:
        part1.append(i)
      else:
        part2.append(i)

  y1,y0 = counter1(nodedata,num_class)
  x11,x10 = counter1(part1,num_class)
  x21,x20 = counter1(part2,num_class)
  if y1==0 or y0==0:
    parent=0
  else:
    parent = -((y0/(y1+y0))*np.log2(y0/(y1+y0))) -((y1/(y1+y0))*np.log2(y1/(y1+y0)))

  if x11==0 or x10==0:
    child1=0
  else:
    child1 = (-((x11/(x11+x10))*np.log2(x11/(x11+x10))) -((x10/(x11+x10))*np.log2(x10/(x11+x10))))

  if x21==0 or x20==0:
    child2=0
  else:
    child2 = (-((x21/(x21+x20))*np.log2(x21/(x21+x20))) -((x20/(x21+x20))*np.log2(x20/(x21+x20))))
  ent = parent -(((x11+x10)/(y1+y0))*child1) -(((x21+x20)/(y1+y0))*child2)
  return ent 

def best_attr(data,conti_att,disc_att,num_class):
  max1=0
  max2=0
  if len(disc_att)>0:
    for i in disc_att:
      ent = Entropy(data,i,1,0.5,num_class)
      if max1 <= ent:
        max1 = ent
        att1=i
  if len(conti_att)>0:
    for x in conti_att:
      ent = Entropy(data,x[0],0,x[1],num_class)
      if max2 <= ent:
        max2 = ent
        att2=x

  if max1+max2==0:
    return 0,0
  else:
    if max1>max2:
      return "disc",att1
    else:
      return "conti",att2
  

In [ ]:
# x>br right and x<br left
class Node:
  def __init__(self, value,num_class):
      self.left = None
      self.data = value
      self.attr = None
      self.right = None
      self.index = None
      self.parent=None
      self.br_point=0.5
      self.disc_att=None
      self.conti_att=None
      self.depth=0
      self.label=None

def decision_tree(train_data,rootdisc_att,rootconti_att,num_class,maxdepth):
  root = Node(train_data,num_class)
  root.disc_att=rootdisc_att
  root.conti_att=rootconti_att


  root.index=0
  nodes=[]
  temp_nodes=[]
  nodes.append(root)
  temp_nodes.append(root)
  while len(temp_nodes)>0:
    curr=temp_nodes.pop()
    if curr.depth == maxdepth:
      for i in curr.data:
        s= all_classes[num_class][i] +s
      if 2*s>len(curr.data):
        nodes[curr.index].label=1
      else:
        nodes[curr.index].label=0
    else:

      if len(curr.data)==0:
        nodes[curr.index].label=nodes[curr.parent].label
      else:
        s=0
        for i in curr.data:
          s= all_classes[num_class][i] +s

        if s==len(curr.data) or s==0:
          nodes[curr.index].label=s
        else:
          typ,ind=best_attr(curr.data,curr.conti_att,curr.disc_att,num_class)

          if typ==0:
            for i in curr.data:
              s= all_classes[num_class][i] +s
            if 2*s>len(curr.data):
              nodes[curr.index].label=1
            else:
              nodes[curr.index].label=0            

          if typ=="conti":

            nodes[curr.index].br_point=ind[1]
            nodes[curr.index].attr=["conti",ind]

            right=[]
            left=[]
            for i in curr.data:
              if float(continuous_data.iloc[i,ind[0]])<float(ind[1]):
                left.append(i)
              else:
                right.append(i)
            
            rnode=Node(right,num_class)
            curr.conti_att.remove(ind)
            rnode.conti_att = curr.conti_att
            rnode.disc_att = curr.disc_att
            rnode.depth=curr.depth+1
            rnode.parent=curr.index

            lnode=Node(left,num_class)
            lnode.conti_att = curr.conti_att
            lnode.disc_att = curr.disc_att
            lnode.depth=curr.depth+1
            lnode.parent=curr.index
            
            nodes.append(rnode)
            nodes.append(lnode)
            nodes[curr.index].right=len(nodes)-2
            nodes[curr.index].left=len(nodes)-1
            nodes[len(nodes)-2].index=len(nodes)-2
            nodes[len(nodes)-1].index=len(nodes)-1 
            
            temp_nodes.append(nodes[len(nodes)-2])
            temp_nodes.append(nodes[len(nodes)-1])  
      
          if typ=="disc":
            nodes[curr.index].br_point=0.5
            nodes[curr.index].attr=["disc",ind]

            right=[]
            left=[]
            for i in curr.data:
              if disc_data.iloc[i,ind]<0.5:
                left.append(i)
              else:
                right.append(i)
            
            rnode=Node(right,num_class)
            curr.disc_att.remove(ind)
            rnode.disc_att = curr.disc_att
            rnode.conti_att = curr.conti_att
            rnode.depth=curr.depth+1

            lnode=Node(left,num_class)
            lnode.disc_att = curr.disc_att
            lnode.conti_att = curr.conti_att
            lnode.depth=curr.depth+1

            nodes.append(rnode)
            nodes.append(lnode)
            nodes[curr.index].right=len(nodes)-2
            nodes[curr.index].left=len(nodes)-1
            nodes[len(nodes)-2].index=len(nodes)-2
            nodes[len(nodes)-1].index=len(nodes)-1 
            
            temp_nodes.append(nodes[len(nodes)-2])
            temp_nodes.append(nodes[len(nodes)-1])

  return nodes


using the tree

In [ ]:
def test_func(all_trees,test):
  TM_TNM_FM_FNM=[]
  conf_mat={}
  for c in classes:
    conf_mat[c]=[0,0,0,0]
  for i in test:
    TM_TNM_FM_FNM.append([0,0,0,0])
    for c in classes:
      path=all_trees[c][0]
      while path.label==None:

        if path.attr[0]=="conti":
          if float(continuous_data.iloc[i,path.attr[1][0]]) > float(path.attr[1][1]):
            path=all_trees[c][path.right]
          else:
            path=all_trees[c][path.left]
        
        else:
          if float(disc_data.iloc[i,path.attr[1]]) > 0.5:
            path=all_trees[c][path.right]
          else:
            path=all_trees[c][path.left]
      
      if path.label==0:
        if fdata.iloc[i,-1]!=c:
          #print(i,"true not mine")
          conf_mat[c][1] = conf_mat[c][1]+1
          TM_TNM_FM_FNM[len(TM_TNM_FM_FNM)-1][1] = TM_TNM_FM_FNM[len(TM_TNM_FM_FNM)-1][1] +1
        else:
          #print(i,"false not mine")
          conf_mat[c][3] = conf_mat[c][3]+1
          TM_TNM_FM_FNM[len(TM_TNM_FM_FNM)-1][3] = TM_TNM_FM_FNM[len(TM_TNM_FM_FNM)-1][3] +1
      else:
        if fdata.iloc[i,-1]==c:
          #print(i,"true  mine")
          conf_mat[c][0] = conf_mat[c][0]+1
          TM_TNM_FM_FNM[len(TM_TNM_FM_FNM)-1][0] = TM_TNM_FM_FNM[len(TM_TNM_FM_FNM)-1][0] +1
        else:
          #print(i,"false mine")
          conf_mat[c][2] = conf_mat[c][2]+1
          TM_TNM_FM_FNM[len(TM_TNM_FM_FNM)-1][2] = TM_TNM_FM_FNM[len(TM_TNM_FM_FNM)-1][2] +1
  return TM_TNM_FM_FNM,conf_mat


In [ ]:
all_trees={}
train_data=list(range(0,50))
disc_attr=list(range(0,disc_data.shape[1]))
test=list(range(51,fdata.shape[0]))
for num_class in range(len(classes)):
  conti_attr=list(enumerate(break_points[num_class]))
  all_trees[classes[num_class]] = decision_tree(train_data , disc_attr , conti_attr , num_class,6)


out,conf_mat=test_func(all_trees,test)
right=0
for r in out:
  if r[0]==1:
    right = right + (1/(1+r[2]))


print("percision of single tree: ",right/len(test))
print(" ")
print("confision matrix for every data: ")
print("True_Mine, True_Not_Mine, False_Mine, False_Not_Mine")
out

percision of single tree:  0.7254901960784315
 
confision matrix for every data: 
True_Mine, True_Not_Mine, False_Mine, False_Not_Mine


[[1, 10, 0, 0],
 [1, 10, 0, 0],
 [0, 10, 0, 1],
 [1, 10, 0, 0],
 [1, 10, 0, 0],
 [1, 9, 1, 0],
 [1, 9, 1, 0],
 [1, 10, 0, 0],
 [1, 10, 0, 0],
 [1, 10, 0, 0],
 [0, 10, 0, 1],
 [1, 10, 0, 0],
 [1, 10, 0, 0],
 [0, 9, 1, 1],
 [1, 8, 2, 0],
 [1, 10, 0, 0],
 [1, 10, 0, 0]]

In [ ]:
print("confision matrix for every tree: ")
print("True_Mine, True_Not_Mine, False_Mine, False_Not_Mine")
conf_mat

confision matrix for every tree: 
True_Mine, True_Not_Mine, False_Mine, False_Not_Mine


{1: [5, 12, 0, 0],
 2: [2, 14, 1, 0],
 3: [1, 16, 0, 0],
 4: [0, 16, 0, 1],
 5: [1, 16, 0, 0],
 6: [0, 15, 2, 0],
 8: [0, 16, 0, 1],
 9: [4, 13, 0, 0],
 10: [0, 16, 0, 1],
 14: [0, 15, 2, 0],
 16: [1, 16, 0, 0]}

In [ ]:
"""
for att in all_trees[2]:
  if att.attr!=None:
    print(att.attr)
"""

Bagging

In [ ]:
bagging_data=[]
for i in range(2):
  bagging_data.append(list(range(25*i,25*(i+1))))

train_data=list(range(0,50))
disc_attr=list(range(0,disc_data.shape[1]))
test=list(range(51,fdata.shape[0]))
bag_out={}
for x in test:
  bag_out[x]=[0]*len(classes)

for j in range(2):
  for num_class in range(len(classes)):
    conti_attr=list(enumerate(break_points[num_class]))
    all_trees[classes[num_class]] = decision_tree(bagging_data[j] , disc_attr , conti_attr , num_class,6)
  for i in test:
    n=0
    for c in classes:
      path=all_trees[c][0]
      while path.label==None:
        if path.attr[0]=="conti":
          if float(continuous_data.iloc[i,path.attr[1][0]]) > float(path.attr[1][1]):
            path=all_trees[c][path.right]
          else:
            path=all_trees[c][path.left]
        
        else:
          if float(disc_data.iloc[i,path.attr[1]]) > 0.5:
            path=all_trees[c][path.right]
          else:
            path=all_trees[c][path.left]
      
      if path.label!=0:
        path.label=1
        
      bag_out[i][n] = bag_out[i][n] + path.label
      n = n+1

In [ ]:
count_bag=0
for x in bag_out:
  c=bag_out[x].index(max(bag_out[x]))
  if fdata.iloc[x,-1]==classes[c]:
    count_bag = count_bag + 1 
print("percision of bagging: %",100*count_bag/len(test))

percision of bagging: % 76.47058823529412


In [ ]:
l = list(range(0, continuous_data.shape[1]-1))
random.shuffle(l)
ind=20*np.array(list(range(0,10)))
con_atts=[]
for i in range(len(ind)):
  if i == len(ind)-1:
    con_atts.append(l[ind[i]:])
  else:
    con_atts.append(l[ind[i]:ind[i+1]])

l = list(range(0, disc_data.shape[1]-1))
random.shuffle(l)
ind=9*np.array(list(range(0,10)))
disc_atts=[]
for i in range(len(ind)):
  if i == len(ind)-1:
    disc_atts.append(l[ind[i]:])
  else:
    disc_atts.append(l[ind[i]:ind[i+1]])

In [ ]:

train_data=list(range(0,50))
test=list(range(51,fdata.shape[0]))

for j in range(10):
  for num_class in range(len(classes)):
    conti_attr=list(enumerate(break_points[num_class]))
    conti_attr=pd.Series(conti_attr)
    conti_attr=list(conti_attr[con_atts[j]])
    disc_attr=disc_atts[j]
    all_trees[classes[num_class]] = decision_tree(train_data , disc_attr , conti_attr , num_class,4)
  for i in test:
    n=0
    for c in classes:
      path=all_trees[c][0]
      while path.label==None:
        if path.attr[0]=="conti":
          if float(continuous_data.iloc[i,path.attr[1][0]]) > float(path.attr[1][1]):
            path=all_trees[c][path.right]
          else:
            path=all_trees[c][path.left]
        
        else:
          if float(disc_data.iloc[i,path.attr[1]]) > 0.5:
            path=all_trees[c][path.right]
          else:
            path=all_trees[c][path.left]
      
      if path.label!=0:
        path.label=1
        
      bag_out[i][n] = bag_out[i][n] + path.label
      n = n+1

In [ ]:
count_bag=0
for x in bag_out:
  c=bag_out[x].index(max(bag_out[x]))
  if fdata.iloc[x,-1]==classes[c]:
    count_bag = count_bag + 1 
print("percision of random_forest: %",100*count_bag/len(test))

percision of random_forest: % 70.58823529411765
